# IRWA Project Part 2

|Name | Email | UPF uNum |
| --- | --- | --- |
| Clara Pena | clara.pena01@estudiant.upf.edu | u186416 |
| Yuyan Wang | yuyan.wang01@estudiant.upf.edu | u199907 |

## Import Libraries and Load Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/processed_data.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117405 entries, 0 to 117404
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        117405 non-null  int64 
 1   content   117404 non-null  object
 2   date      117405 non-null  object
 3   hashtags  116794 non-null  object
 4   likes     117405 non-null  int64 
 5   retweets  117405 non-null  int64 
 6   url       117405 non-null  object
 7   language  117405 non-null  object
 8   docId     48427 non-null   object
dtypes: int64(3), object(6)
memory usage: 8.1+ MB


In [3]:
df.head(10)

,id,content,date,hashtags,likes,retweets,url,language,docId
0,1364506195453767680,ਆ ਚ farmersprotest,2021-02-24T09:23:22+00:00,#FarmersProtest,0,0,https://twitter.com/parmarmaninder/status/1364...,pa,NaN
1,1364506167226032128,watch full video farmersprotest nofarmersnofoo...,2021-02-24T09:23:16+00:00,#farmersprotest #NoFarmersNoFood,0,0,https://twitter.com/anmoldhaliwal/status/13645...,en,doc_2
2,1364506144002088963,kisanektamorcha farmersprotest nofarmersnofood,2021-02-24T09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoFood,0,0,https://twitter.com/KotiaPreet/status/13645061...,en,NaN
3,1364506120497360896,jai jwaan jai kis farmersprotest modiignoringf...,2021-02-24T09:23:05+00:00,#FarmersProtest #ModiIgnoringFarmersDeaths,0,0,https://twitter.com/babli_708/status/136450612...,fi,NaN
4,1364506076272496640,farmersprotest,2021-02-24T09:22:54+00:00,#FarmersProtest,0,0,https://twitter.com/Varinde17354019/status/136...,en,NaN
5,1364505995859423234,modidontsellfarm farmersprotest,2021-02-24T09:22:35+00:00,#ModiDontSellFarmers #FarmersProtest,0,0,https://twitter.com/BitnamSingh/status/1364505...,en,NaN
6,1364505991887347714,watch full video farmersprotest nofarmersnofood,2021-02-24T09:22:34+00:00,#farmersprotest #NoFarmersNoFood,0,0,https://twitter.com/anmoldhaliwal/status/13645...,en,doc_3
7,1364505896576053248,farmersprot,2021-02-24T09:22:11+00:00,#FarmersProtest,0,0,https://twitter.com/SatThiara/status/136450589...,de,NaN
8,1364505892612268032,wheatfield farmersprotest tcoyxsvdprpgg,2021-02-24T09:22:10+00:00,#WheatField #FarmersProtest,1,1,https://twitter.com/PasumaiVikatan/status/1364...,ta,NaN
9,1364505813834989568,watch full video farmersprotest nofarmersnofood,2021-02-24T09:21:51+00:00,#farmersprotest #NoFarmersNoFood,0,0,https://twitter.com/anmoldhaliwal/status/13645...,en,doc_4


## Indexing